In [1]:
#Importing all the neccessary libraries
%matplotlib inline
import sqlite3
import pandas as pd
import numpy as np
import nltk
import string
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.feature_extraction.text import TfidfTransformer, TfidfVectorizer, CountVectorizer
import sklearn.metrics as metrics
from sklearn.metrics import confusion_matrix, roc_curve, auc, precision_score, recall_score, f1_score
from nltk.stem.porter import PorterStemmer
import string
from nltk.corpus import stopwords
from nltk.stem.wordnet import WordNetLemmatizer
#from gensim.models import Word2Vec, KeyedVectors
import pickle
import warnings
warnings.filterwarnings("ignore")
from sklearn import datasets, neighbors
from sklearn.model_selection import train_test_split, cross_val_score
from sklearn.neighbors import KNeighborsClassifier
from sklearn.metrics import accuracy_score
from collections import Counter
from matplotlib.colors import ListedColormap
#import scikitplot.metrics as sciplot
from sklearn.metrics import accuracy_score
import math
import nltk
from sklearn.naive_bayes import MultinomialNB
from sklearn.model_selection import TimeSeriesSplit, GridSearchCV
from nltk.stem import WordNetLemmatizer
nltk.download('wordnet')

[nltk_data] Downloading package wordnet to
[nltk_data]     C:\Users\devan\AppData\Roaming\nltk_data...
[nltk_data]   Package wordnet is already up-to-date!


True

NLP -- natural language processing

English

Whenever u r dealing with text data that will be called as natural languag eprocessing

As of now in all our projects, we hace seen numerical data(int data, float data), categorical data(object data), datetime data(date data)

But now we are going to see text data(string data--- example: reviews, comments, feedbacks, tweets, etc)

How is ur product and how r u satisfied wit the product:

Reviews, comments, feedbacks, tweets, etc -- is our text datasets

Sentiment Analysis: meaninig -- finding the sentiment of the text data --- whether the text data is positive, negative or neutral


The benefit of sentiment analysis is that it helps businesses understand customer opinions, improve products and services, and make informed decisions based on customer feedback.



##### Use below link to download the dataset
##### https://www.kaggle.com/datasets/snap/amazon-fine-food-reviews

#### The immediate code block below does the following things :

1. Load the Amazon dataset.
2. Classify the reviews initially based on their score rating and give them a 'Positve' or a 'Negative' tag.
3. Remove duplicate/redundant datas.
4. Get an idea of how much percentage data were actually duplicates.
5. Plot a histogram which will display the distribution of the number of positive and negative reviews after de-duplication.

###### NOTE : If we dont' clean the data and feed them to an ML system, it basically means we are throwing in a lot of garbage data to the ML system. If we give it garbage, it will give us garbage back. So it's utmost important to clean the data before proceeding.

In [2]:
filtered_data=pd.read_csv("Reviews.csv")

In [3]:
filtered_data.head()

,Id,ProductId,UserId,ProfileName,HelpfulnessNumerator,HelpfulnessDenominator,Score,Time,Summary,Text
0,1,B001E4KFG0,A3SGXH7AUHU8GW,delmartian,1,1,5,1303862400,Good Quality Dog Food,I have bought several of the Vitality canned d...
1,2,B00813GRG4,A1D87F6ZCVE5NK,dll pa,0,0,1,1346976000,Not as Advertised,Product arrived labeled as Jumbo Salted Peanut...
2,3,B000LQOCH0,ABXLMWJIXXAIN,"Natalia Corres ""Natalia Corres""",1,1,4,1219017600,"""Delight"" says it all",This is a confection that has been around a fe...
3,4,B000UA0QIQ,A395BORC6FGVXV,Karl,3,3,2,1307923200,Cough Medicine,If you are looking for the secret ingredient i...
4,5,B006K2ZZ7K,A1UQRSCLF8GW1T,"Michael D. Bigham ""M. Wassir""",0,0,5,1350777600,Great taffy,Great taffy at a great price. There was a wid...


In [4]:
filtered_data.shape

(568454, 10)

In [5]:
filtered_data = filtered_data.sample(frac=0.1, random_state=1)

score ranges from 1 to 5

below 3 -- negative review
above 3 -- positive review
equal to 3 -- neutral review

What if i don't have scores in my dataset?
Based on text data: whether the text data is positive, negative or neutral
Key words from the text data:
How to identify positive, negative and neutral words from the text data?
positive words: good, excellent, nice, amazing, fantastic, love, great, satisfied, happy, wonderful -- in one dictionary
Negative words: bad, poor, terrible, awful, hate, worst, disappointed, unsatisfied, unhappy -- in another dictionary

There may be a chnance that some words may be present in both the dictionaries:
How to deal with this: we will give more priority to negative words than positive words

What is naother way to identify positive, negative and neutral words from the text data?
We can use some pre defined libraries in python:nltk -- natural language toolkit    

Input text data --- I can build any clustering algorithm(three clusters) to identify positive, negative and neutral words from the text data

I am very bad satsfied with the product -- he will give rating as 4    

Sarcasm -- he will give rating as 4 but the review is negative

example for sarcasm: The product is good but the service is bad    -- TATA cars --- 

With in the tool we need to identify the positive, negative and neutral , sarcasm words from the text data

In [6]:
filtered_data.shape

(56845, 10)

In [7]:
filtered_data.head()

,Id,ProductId,UserId,ProfileName,HelpfulnessNumerator,HelpfulnessDenominator,Score,Time,Summary,Text
288312,288313,B000ENUC3S,AN66F3Q4QNU43,Donna Speaker,0,0,5,1340496000,Cherry Pie Larabar,I love the Cherry Pie Lara bar. Best and tast...
431726,431727,B002TMV3CG,A3G007LQX6KGOD,SevereWX,0,0,5,1287878400,Melitta Coffee,Melitta Cafe COllection Blanc et Noir coffee h...
110311,110312,B004867T24,A11LNY2OLQSUSV,M. Castillo,0,0,5,1331769600,great treat,my girls absolutely loved this tuna. they were...
91855,91856,B004U7KPY0,A1QCYVHWO5934U,"PistolaMia ""PistolaMia""",0,0,5,1332806400,Daily Calming,The vendor is fast and dependable. The tea is ...
338855,338856,B000FD78R0,A30U2QQN2FFHE9,J. Amicucci,2,3,5,1271376000,Best Canned Artichokes Out There!,UPDATE - 8/9/2010<br />A lot can happen in jus...


In [8]:
#Give reviews with Score > 3 a 'Positive' tag, and reviews with a score < 3 a 'Negative' tag.
filtered_data['SentimentPolarity'] = filtered_data['Score'].apply(lambda x : 'Positive' if x > 3 else 'Negative')

In [9]:
filtered_data.head(2)

,Id,ProductId,UserId,ProfileName,HelpfulnessNumerator,HelpfulnessDenominator,Score,Time,Summary,Text,SentimentPolarity
288312,288313,B000ENUC3S,AN66F3Q4QNU43,Donna Speaker,0,0,5,1340496000,Cherry Pie Larabar,I love the Cherry Pie Lara bar. Best and tast...,Positive
431726,431727,B002TMV3CG,A3G007LQX6KGOD,SevereWX,0,0,5,1287878400,Melitta Coffee,Melitta Cafe COllection Blanc et Noir coffee h...,Positive


In [10]:
filtered_data['SentimentPolarity'].value_counts()

SentimentPolarity
Positive    44391
Negative    12454
Name: count, dtype: int64

In [11]:

#score greater than 3 as postive  and equal to 3 as neutral and less than 3 as negative
#filtered_data['SentimentPolarity'] = filtered_data['Score'].apply(lambda x : 'Positive' if x > 3 else (if x < 3 else 'Neutral'))
#score greater than 3 as postive  and equal to 3 as neutral and less than 3 as negative and give more priority to negative words
#filtered_data['SentimentPolarity'] = filtered_data['Score'].apply(lambda x : 'Positive' if x > 3 else ('Negative' if x < 3 else 'Neutral'))
filtered_data['Class_Labels'] = filtered_data['SentimentPolarity'].apply(lambda x : 1 if x == 'Positive' else 0)

In [12]:
filtered_data.head(2)

,Id,ProductId,UserId,ProfileName,HelpfulnessNumerator,HelpfulnessDenominator,Score,Time,Summary,Text,SentimentPolarity,Class_Labels
288312,288313,B000ENUC3S,AN66F3Q4QNU43,Donna Speaker,0,0,5,1340496000,Cherry Pie Larabar,I love the Cherry Pie Lara bar. Best and tast...,Positive,1
431726,431727,B002TMV3CG,A3G007LQX6KGOD,SevereWX,0,0,5,1287878400,Melitta Coffee,Melitta Cafe COllection Blanc et Noir coffee h...,Positive,1


In [13]:
'''
# Create dictionaries of positive and negative words from the 'Text' column
from collections import defaultdict
import re

revie1: the product is Great    --- positive
review2: the product is not Great   --- negative label

Algorithm to create the labels: clustering algorithms -- we don't need to create any labels

positive_words_dict = defaultdict(int)
negative_words_dict = defaultdict(int)

for idx, row in filtered_data.iterrows():
    # Clean and tokenize the review text
    words = re.findall(r'\b\w+\b', str(row['Text']).lower())
    if row['SentimentPolarity'] == 'Positive':
        for word in words:
            positive_words_dict[word] += 1
    elif row['SentimentPolarity'] == 'Negative':
        for word in words:
            negative_words_dict[word] += 1

# Convert defaultdict to regular dict for display
positive_words_dict = dict(positive_words_dict)
negative_words_dict = dict(negative_words_dict)

print('Sample positive words:', list(positive_words_dict.keys())[:20])
print('Sample negative words:', list(negative_words_dict.keys())[:20])
'''

"\n# Create dictionaries of positive and negative words from the 'Text' column\nfrom collections import defaultdict\nimport re\n\nrevie1: the product is Great    --- positive\nreview2: the product is not Great   --- negative label\n\nAlgorithm to create the labels: clustering algorithms -- we don't need to create any labels\n\npositive_words_dict = defaultdict(int)\nnegative_words_dict = defaultdict(int)\n\nfor idx, row in filtered_data.iterrows():\n    # Clean and tokenize the review text\n    words = re.findall(r'\x08\\w+\x08', str(row['Text']).lower())\n    if row['SentimentPolarity'] == 'Positive':\n        for word in words:\n            positive_words_dict[word] += 1\n    elif row['SentimentPolarity'] == 'Negative':\n        for word in words:\n            negative_words_dict[word] += 1\n\n# Convert defaultdict to regular dict for display\npositive_words_dict = dict(positive_words_dict)\nnegative_words_dict = dict(negative_words_dict)\n\nprint('Sample positive words:', list(posit

In [14]:
filtered_data.shape

(56845, 12)

In [15]:
print("The number of positive and negative reviews before the removal of duplicate data.")
print(filtered_data["SentimentPolarity"].value_counts())

The number of positive and negative reviews before the removal of duplicate data.
SentimentPolarity
Positive    44391
Negative    12454
Name: count, dtype: int64


In [16]:
#Removing duplicate entries based on past knowledge.
filtered_duplicates=filtered_data.drop_duplicates(subset={"UserId","ProfileName","Time","Text"}, keep='first', inplace=True)

In [17]:
print("The number of positive and negative reviews after the removal of duplicate data.")
print(filtered_data["SentimentPolarity"].value_counts())

The number of positive and negative reviews after the removal of duplicate data.
SentimentPolarity
Positive    40741
Negative    11450
Name: count, dtype: int64


In [18]:
#Removing the entries where HelpfulnessNumerator > HelpfulnessDenominator.
final_data=filtered_data[filtered_data.HelpfulnessNumerator <= filtered_data.HelpfulnessDenominator]

In [19]:
final_data["SentimentPolarity"].value_counts()

SentimentPolarity
Positive    40741
Negative    11450
Name: count, dtype: int64

#### In this code block :

1. I am creating a copy of the final_data dataset called 'sampled_dataset' by dropping the unwanted columns that we don't need for this problem.
2. Sorting the data according to time, such that the oldest reviews are displayed at the top and the latest reviews are displayed at the bottom.
3. Displaying information about the number of postive and negative reviews in the sampled dataset, using a Histogram.

In [20]:
#Dropping unwanted columns for now.
sampled_dataset=final_data.drop(labels=['Id','ProductId', 'UserId', 'Score', 'ProfileName','HelpfulnessNumerator', 'HelpfulnessDenominator','Summary'], axis=1)
print("The shape of the sampled dataset after dropping unwanted columns : ", sampled_dataset.shape)
sampled_dataset.head()

The shape of the sampled dataset after dropping unwanted columns :  (52191, 4)


,Time,Text,SentimentPolarity,Class_Labels
288312,1340496000,I love the Cherry Pie Lara bar. Best and tast...,Positive,1
431726,1287878400,Melitta Cafe COllection Blanc et Noir coffee h...,Positive,1
110311,1331769600,my girls absolutely loved this tuna. they were...,Positive,1
91855,1332806400,The vendor is fast and dependable. The tea is ...,Positive,1
338855,1271376000,UPDATE - 8/9/2010<br />A lot can happen in jus...,Positive,1


In [21]:
#Sorting data according to Time in ascending order => Time Based Splitting Step 1.
sampled_dataset=sampled_dataset.sort_values('Time', axis=0, ascending=False, inplace=False, kind='quicksort', na_position='last')

In [22]:
sampled_dataset = sampled_dataset.reset_index()

In [23]:
sampled_dataset.head(2)

,index,Time,Text,SentimentPolarity,Class_Labels
0,429990,1351209600,Great flavor.....lotsa &#34;heat&#34;....I use...,Positive,1
1,516146,1351209600,I was skeptical about ordering what with the t...,Negative,0


In [24]:
sampled_dataset=sampled_dataset.drop(labels=['index'], axis=1)

In [25]:
#Display distribution of Postive and Negative reviews in a bar graph
sampled_dataset["SentimentPolarity"].value_counts().plot(kind='bar',color=['green','red'],title='Distribution Of Positive and Negative reviews after De-Duplication.',figsize=(5,5))

<Axes: title={'center': 'Distribution Of Positive and Negative reviews after De-Duplication.'}, xlabel='SentimentPolarity'>

#### In this code block :

1. We define two functions which will remove the HTML tags and punctuations from each review.
2. At the end of this code block, each review will contain texts which will only contain alphabetical strings. 
3. We will apply techniques such as stemming and stopwords removal.
3. We will create two columns in the sampled dataset - 'CleanedText' and 'RemovedHTML'.
4. 'CleanedText' column will basically contain the data corpus after stemming the each reviews and removing stopwords from each review. We will use this for our Bag of Word model.
5. 'RemovedHTML' column will contain the data corpus from which only the HTML tags and punctuations are removed. We will use this column for our TF-IDF model, Average Word2Vec model and TF-IDF weighted average Word2Vec model.
6. Store the final table in a dataset called 'sampled_dataset' for future use.

In [26]:
import re


review = "i am satisfied with the product , and the service is good... @  ! 7 % $ and the delivery is on time <br /> <br /> overall i am happy with the product.product"


'''Data Cleaning Stage. Clean each review from the sampled Amazon Dataset.'''
#Data Cleaning Stage. Clean each review from the sampled Amazon Dataset

#Function to clean html tags from a sentence
def removeHtml(sentence): 
    pattern = re.compile('<.*?>')
    cleaned_text = re.sub(pattern,' ',sentence)
    return cleaned_text


In [27]:
cleaned_review=removeHtml(review)
print("The review after removing html tags : ",cleaned_review)

The review after removing html tags :  i am satisfied with the product , and the service is good... @  ! 7 % $ and the delivery is on time     overall i am happy with the product.product


In [28]:
#Function to keep only words containing letters A-Z and a-z. This will remove all punctuations, special characters etc.
def removePunctuations(sentence):
    cleaned_text  = re.sub('[^a-zA-Z]',' ',sentence)
    return cleaned_text

In [29]:
cleaned_review_only_text = removePunctuations(cleaned_review)
print("The review after removing  Punctuations : ",cleaned_review_only_text)

The review after removing  Punctuations :  i am satisfied with the product   and the service is good               and the delivery is on time     overall i am happy with the product product


In [30]:
# ineed to strip the text data and convert it into lower case
#Function to convert the entire sentence to lower case and strip the extra spaces.  
def stripLower(sentence):
    cleaned_text = sentence.lower().strip()
    cleaned_text = re.sub(' +', ' ', cleaned_text)
    return cleaned_text

In [31]:
final_cleaned_review = stripLower(cleaned_review_only_text)
print("The review after converting to lower case and stripping extra spaces : ",final_cleaned_review)

The review after converting to lower case and stripping extra spaces :  i am satisfied with the product and the service is good and the delivery is on time overall i am happy with the product product


In [32]:
final_cleaned_review

'i am satisfied with the product and the service is good and the delivery is on time overall i am happy with the product product'

In [33]:
def take_only_distinct_words(sentence):
    word_tokens = sentence.split()
    #print("The word tokens are : ", word_tokens)
    seen = set()
    cleaned_text = []
    for word in word_tokens:
        if word not in seen:
            cleaned_text.append(word)
            seen.add(word)
    #print("The distinct words are : ", cleaned_text)
    cleaned_text = ' '.join(cleaned_text)
    return cleaned_text

cleaned_review_distinct_words = take_only_distinct_words(final_cleaned_review)
print("The review after taking only distinct words : ", cleaned_review_distinct_words)

The review after taking only distinct words :  i am satisfied with the product and service is good delivery on time overall happy


In [34]:
import nltk   #spacy
from nltk.corpus import stopwords
nltk.download('stopwords')
stop_words = set(stopwords.words('english'))

[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\devan\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


In [35]:
stop_words

{'a',
 'about',
 'above',
 'after',
 'again',
 'against',
 'ain',
 'all',
 'am',
 'an',
 'and',
 'any',
 'are',
 'aren',
 "aren't",
 'as',
 'at',
 'be',
 'because',
 'been',
 'before',
 'being',
 'below',
 'between',
 'both',
 'but',
 'by',
 'can',
 'couldn',
 "couldn't",
 'd',
 'did',
 'didn',
 "didn't",
 'do',
 'does',
 'doesn',
 "doesn't",
 'doing',
 'don',
 "don't",
 'down',
 'during',
 'each',
 'few',
 'for',
 'from',
 'further',
 'had',
 'hadn',
 "hadn't",
 'has',
 'hasn',
 "hasn't",
 'have',
 'haven',
 "haven't",
 'having',
 'he',
 "he'd",
 "he'll",
 "he's",
 'her',
 'here',
 'hers',
 'herself',
 'him',
 'himself',
 'his',
 'how',
 'i',
 "i'd",
 "i'll",
 "i'm",
 "i've",
 'if',
 'in',
 'into',
 'is',
 'isn',
 "isn't",
 'it',
 "it'd",
 "it'll",
 "it's",
 'its',
 'itself',
 'just',
 'll',
 'm',
 'ma',
 'me',
 'mightn',
 "mightn't",
 'more',
 'most',
 'mustn',
 "mustn't",
 'my',
 'myself',
 'needn',
 "needn't",
 'no',
 'nor',
 'not',
 'now',
 'o',
 'of',
 'off',
 'on',
 'once',
 'on

In [36]:
# my next step is to remove stop words from the text data
#Function to remove stop words from a sentence  
nltk.download('stopwords')
def removeStopWords(sentence):
    stop_words = set(stopwords.words('english'))
    # need to take all negative words from the stop words in list after that we will remove those words from the stop words
    #print("The stop words are : ", stop_words)
    negative_words = ["not", "no", "nor", "never", "don't", "didn't", "doesn't", "isn't", "wasn't", "shouldn't", "wouldn't", "couldn't", "won't", "haven't", "hasn't", "hadn't", "mightn't", "mustn't", "shan't"]
    #print("The negative words are : ", negative_words)
    stop_words = stop_words - set(negative_words)
    word_tokens = sentence.split()
    cleaned_text = [word for word in word_tokens if not word in stop_words]
    cleaned_text = ' '.join(cleaned_text)
    return cleaned_text

[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\devan\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


In [37]:
rev_after_stopwords = removeStopWords(cleaned_review_distinct_words)

In [38]:
#rev_after_stopwords

#satisfy, satisfied, satisfied, satisfaction, satisfactory, satisfyingly

In [39]:
#example_review_for_stemming = "i am satisfied with the product and the service is good and satisfy overall the satisfaction is good"
rev_test = "satisfy  satisfied  satisfied satisfaction satisfactory satisfyingly"
# next step is to use stemming or  lemmatisation on the text data:
from nltk.stem.porter import PorterStemmer
def stemming(sentence):
    porter_stemmer = PorterStemmer()
    word_tokens = sentence.split()
    cleaned_text = [porter_stemmer.stem(word) for word in word_tokens]
    cleaned_text = ' '.join(cleaned_text)
    return cleaned_text

In [40]:
review_after_stemming = stemming(rev_test)
print("The review after stemming : ",review_after_stemming)

The review after stemming :  satisfi satisfi satisfi satisfact satisfactori satisfyingli


In [41]:
# create a function to use the stemming technique with snowball stemmer
def stemming_with_snowball(sentence):
    from nltk.stem.snowball import SnowballStemmer
    snowball_stemmer = SnowballStemmer(language='english')
    word_tokens = sentence.split()
    cleaned_text = [snowball_stemmer.stem(word) for word in word_tokens]
    cleaned_text = ' '.join(cleaned_text)
    return cleaned_text

In [42]:
review_after_snowball_stemming = stemming_with_snowball(rev_test)
print("The review after snowball stemmer : ",review_after_snowball_stemming)

The review after snowball stemmer :  satisfi satisfi satisfi satisfact satisfactori satisfi


Stemming Vs Lemmatisation Explanation:
Stemming and lemmatization are both techniques used in natural language processing (NLP) to reduce words to their base or root form, but they do so in different ways and with different levels of accuracy.
Stemming:
Stemming is a crude heuristic process that chops off the ends of words in the hope of achieving the correct base form of the word. It often involves removing common prefixes or suffixes. Stemming algorithms, such as the Porter Stemmer or Snowball Stemmer, use simple rules to perform this task.
Lemmatisation:
Lemmatisation, on the other hand, is a more sophisticated process that considers the context and the morphological analysis of words. It involves using a vocabulary and morphological analysis of words to return the base or dictionary form of a word, known as the lemma. Lemmatisation takes into account the part of speech and the meaning of the word, which often results in more accurate base forms. For example, the lemma of "better" is "good," and the lemma of "running" is "run."
Lemmatisation typically requires more computational resources and is slower than stemming due to its complexity and reliance on linguistic knowledge.


Stemming is faster and more efficient, while lemmatization is more accurate and context-aware. The choice between the two depends on the specific requirements of the NLP task at hand.

In [43]:
def lemmatization(sentence):
    lemmatizer = WordNetLemmatizer()
    word_tokens = sentence.split()
    cleaned_text = [lemmatizer.lemmatize(word) for word in word_tokens]
    cleaned_text = ' '.join(cleaned_text)
    return cleaned_text

review_after_lemmatization = lemmatization(rev_test)
print("The review after lemmatization : ",review_after_lemmatization)

The review after lemmatization :  satisfy satisfied satisfied satisfaction satisfactory satisfyingly


In [44]:
sampled_dataset.head(2)

,Time,Text,SentimentPolarity,Class_Labels
0,1351209600,Great flavor.....lotsa &#34;heat&#34;....I use...,Positive,1
1,1351209600,I was skeptical about ordering what with the t...,Negative,0


In [45]:
def data_preprocessing(sentence):
    sentence = removeHtml(sentence)
    sentence = removePunctuations(sentence)
    sentence = stripLower(sentence)
    sentence = take_only_distinct_words(sentence)
    sentence = removeStopWords(sentence)
    sentence = stemming(sentence)
    return sentence
# we will call this function for each and every review in our dataset
sampled_dataset['Cleaned_Text'] = sampled_dataset['Text'].apply(data_preprocessing)

In [46]:
sampled_dataset.head(2)

,Time,Text,SentimentPolarity,Class_Labels,Cleaned_Text
0,1351209600,Great flavor.....lotsa &#34;heat&#34;....I use...,Positive,1,great flavor lotsa heat use cayenn almost ever...
1,1351209600,I was skeptical about ordering what with the t...,Negative,0,skeptic order troubl cup explod find howev ful...


In [47]:
sampled_dataset = sampled_dataset[['Time','Cleaned_Text','Class_Labels']]

In [48]:
print(sampled_dataset.shape)
sampled_dataset.head()

(52191, 3)


,Time,Cleaned_Text,Class_Labels
0,1351209600,great flavor lotsa heat use cayenn almost ever...,1
1,1351209600,skeptic order troubl cup explod find howev ful...,0
2,1351209600,love faucet husband instal one old hous curren...,1
3,1351209600,dog come outsid train look cupboard wait treat...,1
4,1351209600,everi month give three dog two aussi golden fl...,1


In [49]:
sampled_dataset['Class_Labels'].value_counts()

Class_Labels
1    40741
0    11450
Name: count, dtype: int64

In [50]:
#Split the data set into train and test sets. The test set should be unseen. Time Based Splitting Step 2.
#The top old 80% data will get into the train set. The latest 20% data will get into the test set.
def splitting_data(data):
    X = data['Cleaned_Text']
    y = data['Class_Labels']
    return X,y

In [51]:
X,y = splitting_data(sampled_dataset)

In [52]:
X

0        great flavor lotsa heat use cayenn almost ever...
1        skeptic order troubl cup explod find howev ful...
2        love faucet husband instal one old hous curren...
3        dog come outsid train look cupboard wait treat...
4        everi month give three dog two aussi golden fl...
                               ...                        
52186    beetlejuic not movi watch time one funiest mov...
52187    set small new england town tim burton masterpi...
52188    michael keaton alreadi way major star play gho...
52189    beetlejuic awe inspir wonder amus comed romp e...
52190    twist rumplestiskin captur film star michael k...
Name: Cleaned_Text, Length: 52191, dtype: object

In [53]:
y

0        1
1        0
2        1
3        1
4        1
        ..
52186    1
52187    1
52188    1
52189    1
52190    1
Name: Class_Labels, Length: 52191, dtype: int64

10 days data

8 days of data i can take as train data
2 days of data i can take as test data

In [54]:
# taking first 80% data as train data and last 20% data as test data
split = math.floor(0.8*len(X))
print("The split value is : ", split)
X_train = X[0:split,] ; y_train = y[0:split,]

X_test = X[split:,] ; y_test = y[split:,]

The split value is :  41752


In [55]:
print(X_train.shape)
print(X_test.shape)
print(y_train.shape)
print(y_test.shape)

(41752,)
(10439,)
(41752,)
(10439,)


In [56]:
X_train

0        great flavor lotsa heat use cayenn almost ever...
1        skeptic order troubl cup explod find howev ful...
2        love faucet husband instal one old hous curren...
3        dog come outsid train look cupboard wait treat...
4        everi month give three dog two aussi golden fl...
                               ...                        
41747    big quinoa fan decid give kaniwa tri similar m...
41748    good earth restaur tea addict continu onlin pu...
41749    avid tea drinker drink lot green everi day gre...
41750    one smooth rick dark chocol treat gevalia truf...
41751    pro gevalia say dark chocol mean con sinc oz b...
Name: Cleaned_Text, Length: 41752, dtype: object

In [57]:
X_test

41752               order liter ate day pack addict delici
41753    nice light simpl yet flavor dress receiv compl...
41754    vegeta must almost everyth cook ridicul though...
41755    tea great help concentr also tast well especi ...
41756    tri sever frozen commerci prepar gluten free p...
                               ...                        
52186    beetlejuic not movi watch time one funiest mov...
52187    set small new england town tim burton masterpi...
52188    michael keaton alreadi way major star play gho...
52189    beetlejuic awe inspir wonder amus comed romp e...
52190    twist rumplestiskin captur film star michael k...
Name: Cleaned_Text, Length: 10439, dtype: object

In [58]:
y_train

0        1
1        0
2        1
3        1
4        1
        ..
41747    1
41748    1
41749    1
41750    1
41751    1
Name: Class_Labels, Length: 41752, dtype: int64

In [59]:
y_test

41752    1
41753    1
41754    1
41755    1
41756    1
        ..
52186    1
52187    1
52188    1
52189    1
52190    1
Name: Class_Labels, Length: 10439, dtype: int64

Normalisation vs Standaridation
Label Encoder vs One Hot Encoder

Can I use Label Encoder to convert the text data into numerical data?
0
1
2
3
44
5
6
7
78

9
9
9

1,00,000


In [60]:
# need to calculate the total uniwue words in the text data of X_train data
def total_unique_words(corpus):
    vectorizer = CountVectorizer()
    X = vectorizer.fit_transform(corpus)
    unique_words = vectorizer.get_feature_names_out()
    return unique_words, len(unique_words)

In [61]:
# Calculate total unique words in X_train
unique_words, num_unique_words = total_unique_words(X_train)
print("Total unique words:", num_unique_words)
print("Sample unique words:", unique_words[:20])

Total unique words: 25399
Sample unique words: ['aa' 'aaa' 'aaaa' 'aaaaa' 'aaaaaaaaaaaaaaaaaaaargh' 'aaaaaah' 'aaaaallll'
 'aaaahhhhhh' 'aaah' 'aabsolut' 'aacut' 'aafco' 'aah' 'aakaufman'
 'aalmost' 'aamazon' 'aap' 'aargh' 'aaround' 'aarti']


# we need to convert text data into numerical data
# we will use bag of words or tfidf vectorizer to convert text data into numerical data
# bag of words VS TF-IDF
# we will use tfidf vectorizer to convert text data into numerical data

BOW and TF-IDF Explanation:
Bag of Words (BoW) 
and Term Frequency-Inverse Document Frequency (TF-IDF)

With example by taking few sentences as below:
sentence1 = "I love programming in Python"
sentence2 = "Python is a great programming language"
sentence3 = "I enjoy solving problems with Python"

BOW Formulation:
1. Create a vocabulary of unique words from all sentences.
vocabulary = ["I", "love", "programming", "in", "Python", "is", "a", "great", "language", "enjoy", "solving", "problems", "with"]
len(vocabulary) = 13
2. Represent each sentence as a vector based on the frequency of words in the vocabulary.
maintain the count of each word in one dictionary to see the count of each and every words
words_count = {
    "I": 2,
    "love": 1,
    "programming": 2,
    "in": 1,
    "Python": 3,
    "is": 1,
    "a": 1,
    "great": 1,
    "language": 1,
    "enjoy": 1,
    "solving": 1,
    "problems": 1,
    "with": 1
}
# Example BOW vectors
sentence1 = "I love programming in Python"
sentence1_vector = [2, 1, 2, 1, 3, 0, 0, 0, 0, 0, 0, 0, 0]
sentence2 = "Python is a great programming language"
sentence2_vector = [0, 0, 1, 0, 1, 1, 1, 1, 1, 0, 0, 0, 0]
sentence3 = "I enjoy solving problems with Python"
sentence3_vector = [1, 0, 0, 0, 1, 0, 0, 0, 0, 1, 1, 1, 1]

1, 1, 1, 1, 1, 0, 0, 0, 0, 0, 0, 0, 0    1
0, 0, 1, 0, 1, 1, 1, 1, 1, 0, 0, 0, 0    1

TF-IDF Formulation:
1. Calculate Term Frequency (TF) for each word in a sentence.  
TF(word) = (Number of times word appears in a sentence) / (Total number of words in the sentence)
TF("Python", sentence1) = 1/5 = 0.2
TF("programming", sentence1) = 1/5 = 0.2
TF("I", sentence1) = 1/5 = 0.2
TF("love", sentence1) = 1/5 = 0.2
TF("in", sentence1) = 1/5 = 0.2
TF("is", sentence2) = 1/7 = 0.14
TF("great", sentence2) = 1/7 = 0.14 
TF("language", sentence2) = 1/7 = 0.14
TF("a", sentence2) = 1/7 = 0.14
TF("enjoy", sentence3) = 1/6 = 0.17
TF("solving", sentence3) = 1/6 = 0.17
TF("problems", sentence3) = 1/6 = 0.17
TF("with", sentence3) = 1/6 = 0.17
TF("I", sentence3) = 1/6 = 0.17 
TF("Python", sentence3) = 1/6 = 0.17
IDF ( Inverse Document Frequency)
IDF(word) = log_e(Total number of sentences / Number of sentences with the word in it)
IDF("Python") = log_e(3/3) = 0
IDF("programming") = log_e(3/2) = 0.405
IDF("I") = log_e(3/2) = 0.405
IDF("love") = log_e(3/1) = 1.098
IDF("in") = log_e(3/1) = 1.098
IDF("is") = log_e(3/1) = 1.098
IDF("great") = log_e(3/1) = 1.098
IDF("language") = log_e(3/1) = 1.098
IDF("a") = log_e(3/1) = 1.098
IDF("enjoy") = log_e(3/1) = 1.098
IDF("solving") = log_e(3/1) = 1.098
IDF("problems") = log_e(3/1) = 1.098
IDF("with") = log_e(3/1) = 1.098
TF-IDF(word, sentence) = TF(word, sentence) * IDF(word)
sentence1 = "I love programming in Python"
TF-IDF("Python", sentence1) = 0.2 * 0 = 0
TF-IDF("programming", sentence1) = 0.2 * 0.405 = 0.081
TF-IDF("I", sentence1) = 0.2 * 0.405 = 0.081
TF-IDF("love", sentence1) = 0.2 * 1.098 = 0.22
TF-IDF("in", sentence1) = 0.2 * 1.098 = 0.22
sentence1_vector_tf-idf = [0.081, 0.22, 0.081, 0.22, 0, 0, 0, 0, 0, 0, 0, 0, 0]
sentence1_vector_BOW =    [2, 1, 2, 1, 3, 0, 0, 0, 0, 0, 0, 0, 0]
sentence2 = "Python is a great programming language"
TF-IDF("Python", sentence2) = 0.14 * 0 = 0
TF-IDF("is", sentence2) = 0.14 * 1.098 = 0.154
TF-IDF("a", sentence2) = 0.14 * 1.098 = 0.154
TF-IDF("great", sentence2) = 0.14 * 1.098 = 0.154
TF-IDF("programming", sentence2) = 0.14 * 0.405 = 0.057
TF-IDF("language", sentence2) = 0.14 * 1.098 = 0.154
sentence2_vector_tf-idf = [0, 0, 0.057, 0, 0, 0.154, 0.154, 0.154, 0.154, 0, 0, 0, 0]
sentence2_vector_BOW =    [0, 0, 1, 0, 1, 1, 1, 1, 1, 0, 0, 0, 0]
sentence3 = "I enjoy solving problems with Python"
TF-IDF("I", sentence3) = 0.17 * 0.405 = 0.069
TF-IDF("enjoy", sentence3) = 0.17 * 1.098 = 0.187
TF-IDF("solving", sentence3) = 0.17 * 1.098 = 0.187
TF-IDF("problems", sentence3) = 0.17 * 1.098 = 0.187
TF-IDF("with", sentence3) = 0.17 * 1.098 = 0.187
TF-IDF("Python", sentence3) = 0.17 * 0 = 0
sentence3_vector_tf-idf = [0.069, 0, 0, 0, 0, 0, 0, 0, 0, 0.187, 0.187, 0.187, 0.187]   
sentence3_vector_BOW =    [1, 0, 0, 0, 1, 0, 0, 0, 0, 1, 1, 1, 1]
tfidf_vectorizer = TfidfVectorizer()
tfidf_matrix = tfidf_vectorizer.fit_transform([sentence1, sentence2, sentence3])
tfidf_feature_names = tfidf_vectorizer.get_feature_names_out()  



In [62]:
#Create a function to convert text data into numerical data using Bag of words and need to save that count vetorizer object in pickle file for test data conversion
def text_to_bow(corpus):
    cv_object = CountVectorizer()
    X_bow = cv_object.fit_transform(corpus)
    # we need to save this cv_object in pickle file for test data conversion
    # saving the count vectorizer object in pickle file
    with open('count_vectorizer.pkl', 'wb') as f:
        pickle.dump(cv_object, f)
    return X_bow, cv_object


X_bow , cv_object = text_to_bow(X_train)

In [63]:
X_bow_array = X_bow.toarray()

In [64]:
X_bow.shape

(41752, 25399)

In [65]:
X_train

0        great flavor lotsa heat use cayenn almost ever...
1        skeptic order troubl cup explod find howev ful...
2        love faucet husband instal one old hous curren...
3        dog come outsid train look cupboard wait treat...
4        everi month give three dog two aussi golden fl...
                               ...                        
41747    big quinoa fan decid give kaniwa tri similar m...
41748    good earth restaur tea addict continu onlin pu...
41749    avid tea drinker drink lot green everi day gre...
41750    one smooth rick dark chocol treat gevalia truf...
41751    pro gevalia say dark chocol mean con sinc oz b...
Name: Cleaned_Text, Length: 41752, dtype: object

In [66]:
model = MultinomialNB()

In [67]:
# Train the model using the training sets
model.fit(X_bow,y_train)

MultinomialNB()

In [68]:
#Predict the response for test dataset
y_train_pred = model.predict(X_bow)

In [69]:
# Model Accuracy, how often is the classifier correct?
print("Accuracy:",metrics.accuracy_score(y_train, y_train_pred))

Accuracy: 0.888196972600115


In [70]:
# confusion matrix
cm = confusion_matrix(y_train, y_train_pred)
print("Confusion Matrix:\n", cm)
# we need to convert the X_test data into numerical data using the count vectorizer object which

Confusion Matrix:
 [[ 6643  2995]
 [ 1673 30441]]


In [71]:
# classification report
from sklearn.metrics import classification_report
print("Classification Report on train data:\n", classification_report(y_train, y_train_pred))

Classification Report on train data:
               precision    recall  f1-score   support

           0       0.80      0.69      0.74      9638
           1       0.91      0.95      0.93     32114

    accuracy                           0.89     41752
   macro avg       0.85      0.82      0.83     41752
weighted avg       0.88      0.89      0.89     41752



In [72]:
# we need to convert the text data into numerical data by using saved count vectorizer object
# loading the count vectorizer object from pickle file
#Create a function to convert text data into numerical data using Bag of words for test data conversion
def text_to_bow_test(corpus):
    with open('count_vectorizer.pkl', 'rb') as f:
        loaded_cv_object = pickle.load(f)
    X_bow = loaded_cv_object.transform(corpus)
    return X_bow

X_test_bow = text_to_bow_test(X_test)

In [73]:
y_test_pred = model.predict(X_test_bow)

In [74]:
# classiffication report
print("Classification Report on test data:\n", classification_report(y_test, y_test_pred))

Classification Report on test data:
               precision    recall  f1-score   support

           0       0.68      0.52      0.59      1812
           1       0.90      0.95      0.93      8627

    accuracy                           0.88     10439
   macro avg       0.79      0.74      0.76     10439
weighted avg       0.87      0.88      0.87     10439



In [75]:
# now we need to try with decision tree classifier
from sklearn.tree import DecisionTreeClassifier
dt_model = DecisionTreeClassifier()
dt_model.fit(X_bow, y_train)

DecisionTreeClassifier()

In [76]:
y_train_pred_dt = dt_model.predict(X_bow)
y_test_pred_dt = dt_model.predict(X_test_bow)
print("Classification Report on train data using Decision Tree Classifier:\n", classification_report(y_train,y_train_pred_dt))
print("Classification Report on test data using Decision Tree Classifier:\n", classification_report(y_test,y_test_pred_dt))

Classification Report on train data using Decision Tree Classifier:
               precision    recall  f1-score   support

           0       1.00      1.00      1.00      9638
           1       1.00      1.00      1.00     32114

    accuracy                           1.00     41752
   macro avg       1.00      1.00      1.00     41752
weighted avg       1.00      1.00      1.00     41752

Classification Report on test data using Decision Tree Classifier:
               precision    recall  f1-score   support

           0       0.43      0.44      0.44      1812
           1       0.88      0.88      0.88      8627

    accuracy                           0.80     10439
   macro avg       0.66      0.66      0.66     10439
weighted avg       0.80      0.80      0.80     10439



In [77]:
# now we will try on Random Forest Classifier
from sklearn.ensemble import RandomForestClassifier
rf_model = RandomForestClassifier()
rf_model.fit(X_bow, y_train)
y_train_pred_rf = rf_model.predict(X_bow)
y_test_pred_rf = rf_model.predict(X_test_bow)   
print("Classification Report on train data using Random Forest Classifier:\n", classification_report(y_train,y_train_pred_rf))
print("Classification Report on test data using Random Forest Classifier:\n", classification_report(y_test,y_test_pred_rf))

Classification Report on train data using Random Forest Classifier:
               precision    recall  f1-score   support

           0       1.00      1.00      1.00      9638
           1       1.00      1.00      1.00     32114

    accuracy                           1.00     41752
   macro avg       1.00      1.00      1.00     41752
weighted avg       1.00      1.00      1.00     41752

Classification Report on test data using Random Forest Classifier:
               precision    recall  f1-score   support

           0       0.89      0.21      0.34      1812
           1       0.86      0.99      0.92      8627

    accuracy                           0.86     10439
   macro avg       0.87      0.60      0.63     10439
weighted avg       0.86      0.86      0.82     10439



Class label o having 10k rows 
and class label1 having 30k rows
The data set is imbalanced data set
# we need to do balancing the data set using SMOTE or ADASYN or RandomOverSampler or RandomUnderSampler

Over sampling 
In our data
label1 --1000 rows
label0 ---100 rows    * 10  --- 1000 rows it will become

SMoke technique:
SMOKe is synthetic minority oversampling technique
lablel1 --1000rows
label0 --100rows  --- we are adding 900 rows--- no data loss    



Undersampling Tecnique:
lablel1 --100rows  --- we are dropping 900 rows--- data loss is there
label0 --100rows


RandomOverSampler:



### SMOTE (Synthetic Minority Oversampling Technique) Explanation
SMOTE is a popular technique for handling imbalanced datasets, especially in classification problems. When one class (minority) has significantly fewer samples than another (majority), models can become biased toward the majority class. SMOTE helps to balance the dataset by generating synthetic samples for the minority class.

**How SMOTE Works:**
1. For each sample in the minority class, SMOTE selects one or more of its nearest neighbors (typically using Euclidean distance).
2. It then creates new synthetic samples by interpolating between the original sample and its neighbors. This is done by taking the difference between the feature vector of the sample and its neighbor, multiplying it by a random number between 0 and 1, and adding it to the original sample.
3. The result is a new, plausible sample that lies along the line segment between the two points in feature space.

**Benefits:**
- Reduces bias toward the majority class.
- Helps improve model performance on the minority class.
- No information loss, as opposed to undersampling.

**Limitations:**
- Synthetic samples may introduce noise if the minority class is not well represented.
- Works best with continuous features; may not be ideal for categorical data.

**Example:**
Suppose you have 100 samples of class 0 and 10 samples of class 1. SMOTE can generate 90 synthetic samples for class 1, resulting in a balanced dataset with 100 samples for each class.

**Implementation:**
SMOTE is available in Python via the `imblearn` library:
```python
from imblearn.over_sampling import SMOTE
smote = SMOTE()
X_resampled, y_resampled = smote.fit_resample(X, y)
```

In [78]:
# now we will try with Xgboost classifier
import xgboost as xgb
xgb_model = xgb.XGBClassifier()
xgb_model.fit(X_bow, y_train)
y_train_pred_xgb = xgb_model.predict(X_bow)
y_test_pred_xgb = xgb_model.predict(X_test_bow)
print("Classification Report on train data using XGBoost Classifier:\n", classification_report(y_train,y_train_pred_xgb))
print("Classification Report on test data using XGBoost Classifier:\n", classification_report(y_test,y_test_pred_xgb))

Classification Report on train data using XGBoost Classifier:
               precision    recall  f1-score   support

           0       0.90      0.65      0.76      9638
           1       0.90      0.98      0.94     32114

    accuracy                           0.90     41752
   macro avg       0.90      0.82      0.85     41752
weighted avg       0.90      0.90      0.90     41752

Classification Report on test data using XGBoost Classifier:
               precision    recall  f1-score   support

           0       0.73      0.45      0.56      1812
           1       0.89      0.96      0.93      8627

    accuracy                           0.88     10439
   macro avg       0.81      0.71      0.74     10439
weighted avg       0.87      0.88      0.86     10439



Here is an explanation of each parameter in the param_grid for Random Forest hyperparameter tuning:

n_estimators: Number of trees in the forest. More trees can improve performance but increase computation time.

max_depth: Maximum depth of each tree. Controls how deep the tree can grow. None means nodes are expanded until all leaves are pure.

min_samples_split: Minimum number of samples required to split an internal node. Higher values prevent the model from learning overly specific patterns (overfitting).

min_samples_leaf: Minimum number of samples required to be at a leaf node. Useful for smoothing the model, especially for regression.

max_features: Number of features to consider when looking for the best split. 'auto' (all features), 'sqrt' (square root of total features), 'log2' (log base 2 of total features).

bootstrap: Whether bootstrap samples are used when building trees. If False, the whole dataset is used to build each tree.

class_weight: Weights associated with classes. 'balanced' adjusts weights inversely proportional to class frequencies, useful for imbalanced datasets.

criterion: Function to measure the quality of a split. 'gini' for Gini impurity, 'entropy' for information gain.

These parameters help control the complexity, randomness, and bias/variance trade-off of the Random Forest model. Tuning them can improve accuracy and generalization.

In [79]:
#Random Forest Classifier -- is the combination of multiple decision trees

In [ ]:
# We need to do Hyperparameter tuning for all the models to improve the accuracy and other metrics
# We need to do cross validation for all the models to improve the accuracy and other metrics
# first we will do hyperparameter tuning for Random Forest Classifier

from sklearn.model_selection import GridSearchCV
param_grid = {
    'n_estimators': [50, 100, 200],
    'max_depth': [None, 10, 20],
    'min_samples_split': [2, 5, 10],
    'min_samples_leaf': [1, 2, 4],
    'max_features': ['auto', 'sqrt', 'log2'],
    'bootstrap': [True, False],
    'class_weight': [None, 'balanced'],
    'criterion': ['gini', 'entropy']
}

#comb1 -- 50, None, 2, 1, auto, True,None, gini
#comb2 -- 50, None, 2, 1, auto, True,None, entropy

rf_model = RandomForestClassifier()
grid_search = GridSearchCV(estimator=rf_model, param_grid=param_grid, cv=3, n_jobs=-1, verbose=2)
grid_search.fit(X_bow, y_train)

'''
train data -- 30,000 rows
cv = 3
30,000/3 = 10,000 rows in each fold
2fold for training and 1 fold for validation
    
for example if we take CV=5
30,000/5 = 6,000 rows in each fold
4 folds for training and 1 fold for validation

Train data, Validation data, test data
'''

Fitting 3 folds for each of 1944 candidates, totalling 5832 fits


In [80]:
#RandomSearchCv -- it will take random combinations of hyperparameters   -- if i give 300 value -- out of 1944 it will take only 300 combinations randomly selected

In [81]:
grid_search.best_params_

NameError: name 'grid_search' is not defined

In [82]:
rf_model_tuned = RandomForestClassifier(n_estimators=200, max_depth=20, min_samples_split=5, min_samples_leaf=1, max_features='sqrt', bootstrap=True, class_weight=None, criterion='gini')
rf_model_tuned.fit(X_bow, y_train)
y_train_pred_rf_tuned = rf_model_tuned.predict(X_bow)
y_test_pred_rf_tuned = rf_model_tuned.predict(X_test_bow)
print("Classification Report on train data using Tuned Random Forest Classifier:\n", classification_report(y_train,y_train_pred_rf_tuned))
print("Classification Report on test data using Tuned Random Forest Classifier:\n", classification_report(y_test,y_test_pred_rf_tuned))

Classification Report on train data using Tuned Random Forest Classifier:
               precision    recall  f1-score   support

           0       1.00      0.03      0.05      9638
           1       0.77      1.00      0.87     32114

    accuracy                           0.78     41752
   macro avg       0.89      0.51      0.46     41752
weighted avg       0.83      0.78      0.68     41752

Classification Report on test data using Tuned Random Forest Classifier:
               precision    recall  f1-score   support

           0       1.00      0.00      0.00      1812
           1       0.83      1.00      0.91      8627

    accuracy                           0.83     10439
   macro avg       0.91      0.50      0.45     10439
weighted avg       0.86      0.83      0.75     10439



In [83]:
# With above code still if we are not getting accuracy let us apply Over sampling on the training dataset
from imblearn.over_sampling import RandomOverSampler
ros = RandomOverSampler(random_state=42)
X_resampled, y_resampled = ros.fit_resample(X_bow, y_train)

In [84]:
X_resampled.shape

(64228, 25399)

In [85]:
y_resampled

0        1
1        0
2        1
3        1
4        1
        ..
64223    0
64224    0
64225    0
64226    0
64227    0
Name: Class_Labels, Length: 64228, dtype: int64

In [86]:
# build the random forest model on the resampled data
rf_model_tuned = RandomForestClassifier(n_estimators=200, max_depth=20, min_samples_split=5, min_samples_leaf=1, max_features='sqrt', bootstrap=True, class_weight=None, criterion='gini')
rf_model_tuned.fit(X_resampled, y_resampled)
y_train_pred_rf_tuned = rf_model_tuned.predict(X_bow)
y_test_pred_rf_tuned = rf_model_tuned.predict(X_test_bow)
print("Classification Report on train data using Tuned Random Forest Classifier after OverSampling:\n", classification_report(y_train,y_train_pred_rf_tuned))
print("Classification Report on test data using Tuned Random Forest Classifier after OverSampling:\n", classification_report(y_test,y_test_pred_rf_tuned))

Classification Report on train data using Tuned Random Forest Classifier after OverSampling:
               precision    recall  f1-score   support

           0       0.74      0.83      0.78      9638
           1       0.95      0.91      0.93     32114

    accuracy                           0.89     41752
   macro avg       0.85      0.87      0.86     41752
weighted avg       0.90      0.89      0.90     41752

Classification Report on test data using Tuned Random Forest Classifier after OverSampling:
               precision    recall  f1-score   support

           0       0.58      0.66      0.61      1812
           1       0.93      0.90      0.91      8627

    accuracy                           0.86     10439
   macro avg       0.75      0.78      0.76     10439
weighted avg       0.87      0.86      0.86     10439



In [87]:
# save the rf_model_tuned in pickle file
with open('sentiment_model_bow.pkl', 'wb') as f: 
    pickle.dump(rf_model_tuned, f)

#save the bow vectorizer object in pickle file
with open('vectorizer_bow.pkl', 'wb') as f: 
    pickle.dump(cv_object, f)
